In [2]:
import pandas as pd

# 1. Veriyi pkl dosyasından yükle
file_path = 'hazirlanmis_urun_ozellikleri.pkl'
data = pd.read_pickle(file_path)

# İlk birkaç satırı görüntüle
print("Veri Setindeki İlk 5 Satır:")
print(data.head())

print("\n")

Veri Setindeki İlk 5 Satır:
   urun_id                                                adi marka_id  \
0    16328  MSI PRO B650M-B DDR5 AMD B650 AM5 Soket DDR5 6...       75   
1    90137  MSI Modern 15 F13MG-098XTR i7-1355U 16GB DDR4 ...       75   
2    25777  Sony DualSense Siyah Kablosuz PS5 Oyun Kolu (İ...      107   
3    91026  Sony DualSense Beyaz Kablosuz PS5 Oyun Kolu (İ...      107   
4    11443  Sony PlayStation 5 Slim Digital Sürüm Oyun Kon...      107   

  kategori_id                                        ozellikleri  
0         779  AM5, AMD, AMD B650, 2 x USB 2.0, 4 x USB 2.0, ...  
1         834  Intel Core i7-1355U, 3.70GHz, INTEL Core™ i7, ...  
2        1426                                Siyah, Gamepad, PS5  
3        1426                                Beyaz, Gamepad, PS5  
4         850                1TB, İthalatçı, Playstation 5, Slim  




In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 'ozellikleri' sütununu TF-IDF ile vektörleştirme
vectorizer = TfidfVectorizer(stop_words='english')
feature_matrix = vectorizer.fit_transform(data['ozellikleri'])

# Matrisin boyutlarını kontrol et
print("TF-IDF matris boyutları:", feature_matrix.shape)


TF-IDF matris boyutları: (23386, 10014)


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Veri setinden rastgele 5000 satır seçelim
df_sampled = data.sample(n=5000, random_state=42)

# TF-IDF ve similarity işlemlerini bu alt kümede tekrar yapın
feature_matrix = vectorizer.fit_transform(df_sampled['ozellikleri'])
similarity_matrix = cosine_similarity(feature_matrix, feature_matrix)

print("Alt küme benzerlik matrisi boyutları:", similarity_matrix.shape)


Alt küme benzerlik matrisi boyutları: (5000, 5000)


In [6]:
# Örneklenmiş veri çerçevesini sıfırdan başlayacak şekilde yeniden indeksleyin
df_sampled = df_sampled.reset_index(drop=True)


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# TF-IDF işlemini tekrar yap
vectorizer = TfidfVectorizer(stop_words='english')
feature_matrix = vectorizer.fit_transform(df_sampled['ozellikleri']).toarray()  # Sparse'den dense'e

# Örneklenmiş veri çerçevesini sıfırdan başlayacak şekilde yeniden indeksle
df_sampled = df_sampled.reset_index(drop=True)

# Kullanıcının beğendiği ürünleri belirle (MANUEL)
user_liked_products = [87021]  # Örnek: Buraya kendi ürün ID'nizi yazabilirsiniz
liked_indices = df_sampled[df_sampled['urun_id'].isin(user_liked_products)].index

if len(liked_indices) == 0:
    print(f"Uyarı: Seçilen ürün ID'si ({user_liked_products}) bulunamadı.")
else:
    print("Beğenilen ürünlerin indeksleri:", liked_indices)

    # Kullanıcı profili oluştur
    user_profile = feature_matrix[liked_indices].mean(axis=0)
    print("Kullanıcı Profili Şekli:", user_profile.shape)

    # Kullanıcı profilini TF-IDF matrisiyle karşılaştır ve benzerlik skorlarını hesapla
    user_similarity_scores = cosine_similarity(user_profile.reshape(1, -1), feature_matrix).flatten()
    print("Benzerlik Skorları İlk 10:", user_similarity_scores[:10])

    # Önerilebilecek ürünlerin indekslerini al
    recommendable_indices = np.argsort(user_similarity_scores)[::-1]
    recommendable_indices = [i for i in recommendable_indices if i not in liked_indices]
    recommendable_indices = [i for i in recommendable_indices if 0 <= i < len(df_sampled)]

    print("Önerilebilecek Ürünlerin İndeksleri:", recommendable_indices[:10])

    # İlk 10 öneriyi al
    top_10_indices = recommendable_indices[:10]
    recommendations = df_sampled.iloc[top_10_indices]

    # Kullanıcının beğendiği ürünü göster
    print("\nKullanıcının Beğendiği Ürün:")
    print(df_sampled.loc[liked_indices, ['urun_id', 'adi', 'ozellikleri']])

    # Önerilen ürünleri göster
    print("\nÖnerilen Ürünler:")
    print(recommendations[['urun_id', 'adi', 'ozellikleri']])


Beğenilen ürünlerin indeksleri: Index([6], dtype='int64')
Kullanıcı Profili Şekli: (5362,)
Benzerlik Skorları İlk 10: [0.0893213  0.01841666 0.         0.01470638 0.07446253 0.06340223
 1.         0.02750519 0.04009963 0.01341263]
Önerilebilecek Ürünlerin İndeksleri: [2276, 3204, 3167, 2720, 4693, 4018, 2030, 421, 3006, 472]

Kullanıcının Beğendiği Ürün:
   urun_id                                                adi  \
6    87021  TCL 20L Plus 5G 256GB 6GB RAM Mavi Cep Telefon...   

                                         ozellikleri  
6  2. Sim girişine hafıza kartı veya sim kart tak...  

Önerilen Ürünler:
      urun_id                                                adi  \
2276    87331  Xiaomi Redmi Note 12 Pro 256GB 8GB Kutup Beyaz...   
3204    23149  Samsung Galaxy A53 5G 128GB 8GB RAM Mavi Cep T...   
3167    71148  TCL 20 Pro 5G 256GB 6GB RAM Mavi Cep Telefonu ...   
2720    52128  Xiaomi Redmi 10 2022 64GB Deniz Mavisi Cep Tel...   
4693    48562  Xiaomi Redmi 10 2022 128GB D

In [17]:
# CC (Coverage Criterion) hesaplama fonksiyonu
def calculate_cc(recommendations, total_products):
    # Eğer recommendations bir dict değilse düz liste gibi ele al
    if isinstance(recommendations, dict):
        recommended_items = set()
        for items in recommendations.values():
            recommended_items.update(items)
    else:
        # ndarray, DataFrame, Series vs.
        recommended_items = set(recommendations)

    coverage = len(recommended_items) / len(total_products)
    return coverage

total_products = df_sampled['urun_id'].unique()
cc_score = calculate_cc(recommendations['urun_id'], total_products)  # Eğer df ise
print(f"CC Değeri: {cc_score:.4f}")



CC Değeri: 0.0020


In [20]:
def calculate_map(recommendations, ground_truths):
    import numpy as np
    average_precisions = []

    for user_id, recommended_items in recommendations.items():
        if user_id not in ground_truths or not ground_truths[user_id]:
            continue

        relevant_items = ground_truths[user_id]
        precision_sum = 0.0
        relevant_count = 0

        for i, item in enumerate(recommended_items):
            if item in relevant_items:
                relevant_count += 1
                precision_sum += relevant_count / (i + 1)  # Precision@k

        average_precision = precision_sum / len(relevant_items)
        average_precisions.append(average_precision)

    return np.mean(average_precisions) if average_precisions else 0.0

# Eğer user_id yoksa, otomatik boş dictionary ver
ground_truths = {}


map_score = calculate_map(recommendations, ground_truths)
print(f"MAP Değeri: {map_score:.4f}")



MAP Değeri: 0.0000


In [22]:
# Popülerlik skorlarını veri setinden türetme
def generate_recommendations_and_products(similarity_matrix, df, top_n=5):
    """
    TF-IDF benzerliğine göre ürün-ürün tavsiyeleri oluşturur.

    Returns:
        dict: urun_id → önerilen urun_id listesi
        list: önerilen tüm ürünler
    """
    from sklearn.metrics.pairwise import cosine_similarity
    import numpy as np

    product_ids = df['urun_id'].tolist()
    recommendations = {}

    for idx, product_id in enumerate(product_ids):
        similarity_scores = list(enumerate(similarity_matrix[idx]))
        similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

        top_similar = [
            product_ids[i] for i, score in similarity_scores[1:top_n+1]  # ilk kendisi olduğu için [1:]
        ]

        recommendations[product_id] = top_similar

    # Tek listede tüm önerilen ürünler (MNS için)
    recommended_items_flat = [item for sublist in recommendations.values() for item in sublist]

    return recommendations, recommended_items_flat


# Dinamik MNS hesaplama
def calculate_mns(recommendations, popularity_scores):
    import numpy as np

    novelty_scores = []

    for user_id, recommended_items in recommendations.items():
        user_novelty = []
        for item in recommended_items:
            if item in popularity_scores:
                novelty = 1 / (popularity_scores[item] + 1)  # Popülerlik 0 olursa hata oluşmasın diye +1
                user_novelty.append(novelty)
        if user_novelty:
            novelty_scores.append(np.mean(user_novelty))

    return np.mean(novelty_scores) if novelty_scores else 0.0

# Popülerlik skorlarını türet
# Popülerlik skorlarını türet
popularity_scores = generate_popularity_scores(df_sampled, product_col="urun_id")

# Önerileri oluştur
recommendations, _ = generate_recommendations_and_products(similarity_matrix, df_sampled, top_n=5)

# MNS hesapla
mns_score = calculate_mns(recommendations, popularity_scores)
print(f"MNS Değeri: {mns_score:.4f}")


MNS Değeri: 0.5000


In [23]:
def calculate_mds(recommendations, similarity_matrix, product_indices):

    import numpy as np

    diversity_scores = []

    for user_id, recommended_items in recommendations.items():
        diversity = []
        for i, item1 in enumerate(recommended_items):
            for j, item2 in enumerate(recommended_items):
                if i < j:  # Çiftleri sadece bir kere hesapla
                    idx1 = product_indices.get(item1)
                    idx2 = product_indices.get(item2)
                    if idx1 is not None and idx2 is not None:
                        # Benzerlik değerini al ve 1 - benzerlik ile çeşitlilik hesapla
                        diversity.append(1 - similarity_matrix[idx1, idx2])
        if diversity:
            diversity_scores.append(np.mean(diversity))

    return np.mean(diversity_scores) if diversity_scores else 0.0

# Ürün ID'lerini indekslere eşleyen sözlük oluştur
product_indices = {product_id: idx for idx, product_id in enumerate(df_sampled["urun_id"])}

# MDS hesaplama
mds_score = calculate_mds(recommendations, similarity_matrix, product_indices)

# Sonucu yazdır
print(f"MDS Değeri: {mds_score:.4f}")


MDS Değeri: 0.3301


In [24]:
def calculate_mps(similarity_matrix, product_ids, top_n=5):

    import numpy as np

    # Öneriler oluşturma
    recommendations = {}
    for idx, product_id in enumerate(product_ids):
        similar_indices = similarity_matrix[idx].argsort()[::-1][1:top_n+1]
        recommendations[product_id] = [product_ids[i] for i in similar_indices]

    # Kullanıcılar arası çeşitlilik hesaplama
    user_pairs_diversity = []
    product_keys = list(recommendations.keys())

    for i, user1 in enumerate(product_keys):
        for j, user2 in enumerate(product_keys):
            if i < j:  # Her kullanıcı çiftini sadece bir kez değerlendir
                items_user1 = set(recommendations[user1])
                items_user2 = set(recommendations[user2])

                # Öneri listelerinin farklılık oranını hesapla
                diversity = 1 - (len(items_user1.intersection(items_user2)) / len(items_user1.union(items_user2)))
                user_pairs_diversity.append(diversity)

    # Ortalama çeşitlilik (MPS)
    return np.mean(user_pairs_diversity) if user_pairs_diversity else 0.0

# Ürün ID'lerini liste olarak al
product_ids = df_sampled["urun_id"].tolist()

# MPS hesaplama
mps_score = calculate_mps(similarity_matrix, product_ids, top_n=5)

# Sonucu yazdır
print(f"MPS Değeri: {mps_score:.4f}")


MPS Değeri: 0.9988


In [25]:
import pickle

# Tüm metrikleri tek bir sözlükte topla
contant_based_results = {
    "MAP": map_score,
    "CC": cc_score,
    "MNS": mns_score,
    "MDS": mds_score,
    "MPS": mps_score
}

# Tek bir dosyaya kaydet
with open("contant_based_sonuclari.pkl", "wb") as f:
    pickle.dump(contant_based_results, f)
